In [1]:
import clip
import torch
import numpy as np
from tqdm import tqdm
import os
from PIL import Image

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
device

'cpu'

`device` will indicate wich `CLIP` model to use depending on the available hardware. If there is a GPU, `cuda:0` will be used  or `cuda:1` if there are multiple GPUs. If there is not a GPU, `cpu` will be used.

In [3]:
labels = ['black', 'white', 'asian', 'indian']
tkns = ['A photo of a person of color ' + label for label in labels]
text = clip.tokenize(tkns).to(device)

`tkns` is the domain of possible values. `CLIP` model predicts for each image the most probable sentence from `tkns`, in this case. 

In [4]:
BATCH_SIZE = 100000

dir_path = r'/Users/hanselblanco/Documents/4to/ML/UTKFace/UTKFace'
ln = 0
photo_paths = os.listdir(dir_path)

for path in photo_paths:
    if os.path.isfile(os.path.join(dir_path, path)):
        ln += 1


Initializing usefull variables for `CLIP` model application.

In [5]:
results = []
photos_to_analize = 4000

for i in tqdm(range(0, ln, BATCH_SIZE)):
    images = [preprocess(Image.open(dir_path + '/' + photo_paths[j])) for j in range(photos_to_analize)]
    image_input = torch.tensor(np.stack(images)).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        logits_per_image, logits_per_text = model(image_input, text)
        # The softmax function takes the original confidence and applys a transform to make all the confidence add up to one
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        results.append(probs)

100%|██████████| 1/1 [09:35<00:00, 575.29s/it]


Executing `CLIP` model for `photos_to_analize` images from the dataset.

In [6]:
res = np.concatenate(results,axis=0)
choices = np.argmax(res,axis=1)
choices.shape

(4000,)

In [7]:
getlabel = lambda x:labels[x]
vgetlabel = np.vectorize(getlabel)
colors = vgetlabel(choices)
colors

array(['asian', 'black', 'white', ..., 'white', 'black', 'asian'],
      dtype='<U6')

`colors` is the vector with predicted labels to add to each sentence from `tkns` for each image, ordered.

## Detecting bias in `CLIP` results.

### Selection Rate (Positive results / N)

In [8]:
race_code = { 0 : 'white', 1 : 'black', 2 : 'asian', 3 : 'indian', 4 : 'others'}

positive_whites, positive_blacks, positive_asians, positive_indians = 0, 0, 0, 0

total_whites, total_blacks, total_asians, total_indians = 0, 0, 0, 0

for i in range(photos_to_analize):
    data = photo_paths[i].split('_')
    race_number = int(data[2])
    match race_code[race_number]:
        case 'white':
            total_whites += 1
            if colors[i] == 'white':
                positive_whites += 1
        case 'black':
            total_blacks += 1
            if colors[i] == 'black':
                positive_blacks += 1
        case 'asian':
            total_asians += 1
            if colors[i] == 'asian':
                positive_asians += 1
        case 'indian':
            total_indians += 1
            if colors[i] == 'indian':
                positive_indians += 1
        case default:
            continue
                
whites_sr, blacks_sr, asians_sr, indians_sr = positive_whites/ total_whites, positive_blacks/ total_blacks, positive_asians/ total_asians, positive_indians/ total_indians

whites_sr, blacks_sr, asians_sr, indians_sr
    

(0.9313380281690141, 0.8233082706766918, 0.9378427787934186, 0.8)

#### Disparate impact

In [9]:

# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact_b_w = blacks_sr/ whites_sr
disp_impact_b_a = asians_sr/ whites_sr
disp_impact_b_i = indians_sr/ whites_sr
disp_impact_b_w, disp_impact_b_a, disp_impact_b_i

(0.8840058558494535, 0.8778745108384999, 1.0291353383458646)

In [10]:
if disp_impact_b_w < 0.8:
    print('Disparate impact present in black group / white group')
if disp_impact_b_a < 0.8:
    print('Disparate impact present in asian group / white group')
if disp_impact_b_i < 0.8:
    print('Disparate impact present in indian group / white group')
